In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import preprocessing
import informe
from sklearn.metrics import plot_roc_curve, classification_report

from sklearn.model_selection import GridSearchCV 
from sklearn import svm

sns.set()

In [46]:
GSPREADHSEET_DOWNLOAD_URL = (
    "https://docs.google.com/spreadsheets/d/{gid}/export?format=csv&id={gid}".format
)

FIUFIP_2021_1_GID = '1-DWTP8uwVS-dZY402-dm0F9ICw_6PNqDGLmH0u8Eqa0'
df = pd.read_csv(GSPREADHSEET_DOWNLOAD_URL(gid=FIUFIP_2021_1_GID))

# Preprocesamiento

In [47]:
df = preprocessing.remove_irrelevant_features(df)
df = preprocessing.missings_treatment(df)
df = preprocessing.one_hot_encodding(df)
X = df.drop('tiene_alto_valor_adquisitivo', axis='columns')

# Se separa el dataset en entrenamiento y validación
y = df.tiene_alto_valor_adquisitivo
X_train, X_val, y_train, y_val = preprocessing.dataset_split(X, y, test_size=0.40)

# Se separa el dataset de validacion en 2 partes, X_val para seleccionar el mejor modelo de knn y X_holdout para el test final
X_val, X_holdout, y_val, y_holdout = preprocessing.dataset_split(X_val, y_val, 0.2)

# SVM: entrenamiento y búsqueda de hiperparámetros.

Se usa GridSearch para buscar la mejor combinación de hiperparámetros. Se elige la mejor combinación según su score ROC-AUC. 

In [40]:
params = {'C': [0.1, 1, 10], 'gamma': ['scale', 'auto'], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'degree': [2, 3]} #revisar degree

## Modelo 1

Se entrena un modelo de SVM con los datos originales. Se usan todos los features.

In [5]:
clf = svm.SVC(cache_size = 400)

gscv_lineal = GridSearchCV(
    clf, params, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train, y_train)

gscv_lineal.best_params_

KeyboardInterrupt: 

### Métricas


In [ ]:
informe.imprimir_metricas(gscv_lineal, X_test, y_test, 'SVM con kernel lineal:')
roc = plot_roc_curve(gscv_lineal.best_estimator_, X_test, y_test)

## Modelo 2: datos normalizados

Se entrena un modelo de SVM con los datos numéricos normalizados por . Se usan todos los features.

In [50]:
X_train_n, normalizer = preprocessing.normalizar(X_train)
X_val_n, _ = preprocessing.normalizar(X_val, normalizer)

In [ ]:
clf = svm.SVC(cache_size = 400)

gscv_lineal = GridSearchCV(
    clf, params, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train, y_train)

gscv_lineal.best_params_

### Métricas


In [ ]:
informe.imprimir_metricas(gscv_lineal, X_test, y_test, 'SVM con kernel lineal:')
roc = plot_roc_curve(gscv_lineal.best_estimator_, X_test, y_test)

## Modelo 3: usando pca

Se entrena un modelo de SVM aplicando PCA sobre el dataset para reducir su dimensión.

In [42]:
X_train_pca, pca = preprocessing.pca(X_train, n_components=4)
X_val_pca, _ = preprocessing.pca(X_val, pca=pca)

In [8]:
clf = svm.SVC(cache_size = 400)

gscv_lineal = GridSearchCV(
    clf, params, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train_pca, y_train)

gscv_lineal.best_params_

KeyboardInterrupt: 

### Métricas


In [ ]:
informe.imprimir_metricas(gscv_lineal, X_test, y_test, 'SVM con kernel lineal:')
roc = plot_roc_curve(gscv_lineal.best_estimator_, X_test, y_test)

## Modelo 4: datos normalizados y usando pca

Se entrena un modelo de SVM, normalizando los datos y luego aplicando PCA sobre el dataset para reducir su dimensión.

In [51]:
X_train_pca_n, pca_n = preprocessing.pca(X_train_n, n_components=4)
X_val_pca_n, _ = preprocessing.pca(X_val, pca=pca_n)

In [ ]:
clf = svm.SVC(cache_size = 400)

gscv_lineal = GridSearchCV(
    clf, params, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train_pca_n, y_train)

gscv_lineal.best_params_

### Métricas


In [ ]:
informe.imprimir_metricas(gscv_lineal, X_val_pca_n, y_val, 'SVM con kernel lineal:')
roc = plot_roc_curve(gscv_lineal.best_estimator_, X_val_pca_n, y_val)

# Conclusión
      En base a la metrica AUC-ROC, se elige el modelo de SVM con kernel radial
      
### Informe del modelo usando los datos del test_holdout

In [ ]:
import sklearn

y_pred = gscv_rbf.predict(X_test)

informe.plt_distribucion_de_clases(y_test)
informe.plot_matriz_de_confusion(gscv_rbf, X_test, y_test)
roc = plot_roc_curve(gscv_rbf.best_estimator_, X_test, y_test)
print('Métricas:')
informe.imprimir_metricas(gscv_sig, X_test, y_test, 'SVM con kernel sigmoideo:')

## Conclusiones de las métricas observadas de los datos de test_holdout

- accuracy:

        El modelo clasifica los datos de forma correcta en aproximadamente un 75%, viendo la distribucion de clases de la muestra se observa que el 0 es la clase mayoritaria con una proporción de aproximadamente 76%.
    
    
- precisión:

        La fracción de predicciones de 0's que realmente eran 0's fue de aproximadamente 87% y la fracción de predicciones de 1's que realmente eran 1's fue del 78% 


- recall:

        Los 0's reales detectados fueron aproximadamente el 93%, y los 1's reales detectados fueron del 56%. Viendo este resultado en conjunto con la precisión, se entiende que el modelo es mejor prediciendo los 0's que los 1's 


- f1 score:
   
       La calidad del modelo es de 41% en terminos del recall y la precision asi como el balance entre ambas
       
       
- matriz de confusion:

        Se puede ver que las predicciones mayoritarias caen en la diagonal principal, lo cual es una buena caracteristica de una matriz de confusión. sin embargo para los 1's, las dos columnas estan demasiado balanceadas, lo cual ya se sabia ya que en el resultado de recall para los 1's no era muy bueno.
        
        
- UAC ROC:

        Esta métrica indica que el modelo es bueno distinguiendo clases en un 75%. Este valor será utilizado para decidir sobre la elección de este modelo.


# Final test

In [ ]:
df_test = informe.get_df_test()

#preprocesamiento
df_test = preprocessing.ordinal_encode(df_test)
df_test = preprocessing.dummy_variables(df_test)
df_test, _ = preprocessing.normalizar(df_test, normalizer)
df_test, _ = preprocessing.pca(df_test, pca)

y_pred = gscv_rbf.predict(df_test)
informe.save_pred(y_pred, 'svm')